In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras import regularizers
from sklearn.metrics import roc_curve, auc
from keras.layers import Activation
from keras import backend as K

In [ ]:
# load saved df
df = pd.read_pickle('test.pkl')
print (df.keys())

In [ ]:
# filter events
ieta = 24

df = df.loc[abs(df[b'hcal_ieta']) == ieta]

df = df.loc[df["b'hcal_edepth1'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth1'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth2'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth2'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth3'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth3'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth4'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth4'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth5'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth5'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth6'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth6'_y"] > 0.00001]

if ieta in range (26,28):
    df = df.loc[df["b'hcal_edepth7'_x"] > 0.00001]
    df = df.loc[df["b'hcal_edepth7'_y"] > 0.00001]

In [ ]:
df["b'hcal_edepth1'_x"]=np.log(10000*df["b'hcal_edepth1'_x"])
df["b'hcal_edepth1'_y"]=np.log(10000*df["b'hcal_edepth1'_y"])

df["b'hcal_edepth2'_x"]=np.log(10000*df["b'hcal_edepth2'_x"])
df["b'hcal_edepth2'_y"]=np.log(10000*df["b'hcal_edepth2'_y"])

df["b'hcal_edepth3'_x"]=np.log(10000*df["b'hcal_edepth3'_x"])
df["b'hcal_edepth3'_y"]=np.log(10000*df["b'hcal_edepth3'_y"])

df["b'hcal_edepth4'_x"]=np.log(10000*df["b'hcal_edepth4'_x"])
df["b'hcal_edepth4'_y"]=np.log(10000*df["b'hcal_edepth4'_y"])

df["b'hcal_edepth5'_x"]=np.log(10000*df["b'hcal_edepth5'_x"])
df["b'hcal_edepth5'_y"]=np.log(10000*df["b'hcal_edepth5'_y"])

df["b'hcal_edepth6'_x"]=np.log(10000*df["b'hcal_edepth6'_x"])
df["b'hcal_edepth6'_y"]=np.log(10000*df["b'hcal_edepth6'_y"])

if ieta in range (26,28):
    df["b'hcal_edepth7'_x"]=np.log(10000*df["b'hcal_edepth7'_x"])
    df["b'hcal_edepth7'_y"]=np.log(10000*df["b'hcal_edepth7'_y"])

In [ ]:
# keep originals before scaling
df['hcal_edepth1_un'] = df["b'hcal_edepth1'_x"]
df['hcal_edepth2_un'] = df["b'hcal_edepth2'_x"]
df['hcal_edepth3_un'] = df["b'hcal_edepth3'_x"]
df['hcal_edepth4_un'] = df["b'hcal_edepth4'_x"]
df['hcal_edepth5_un'] = df["b'hcal_edepth5'_x"]
df['hcal_edepth6_un'] = df["b'hcal_edepth6'_x"]
df['hcal_edepth7_un'] = df["b'hcal_edepth7'_x"]

df['hcal_ieta_un'] = df[b'hcal_ieta']

In [ ]:
# scaling input vars
cols_to_minmax =[b'pt_of_muon',b'eta_of_muon',b'phi_of_muon',b'energy_of_muon',b'hcal_ieta',
                b'hcal_iphi',     b'IsolationR04',     b'IsolationR03',
                b'ecal_3into3',      b'hcal_3into3', 
                b'ecal_3x3',         b'hcal_1x1',
               "b'hcal_edepth1'_x", "b'hcal_edepth2'_x", "b'hcal_edepth3'_x",
               "b'hcal_edepth4'_x", "b'hcal_edepth5'_x", "b'hcal_edepth6'_x",
               "b'hcal_edepth7'_x"]


df[cols_to_minmax] = df[cols_to_minmax].apply(lambda x: (x - x.min()) /  (x.max() - x.min()))

In [ ]:
for i in (df.keys()):
    print (i)
    if i == "b'hcal_edepth7'_x":
        continue
    if i == "b'hcal_edepth7'_y":
        continue
    if i == "b'hcal_edepth7'_un":
        continue
    plt.hist(df[i],bins=100,label=str(i),alpha=0.4)
    plt.yscale("log")
    plt.legend()
    plt.show()

In [ ]:
data = df.values
print (data)
ntest = 20000
testindx = data.shape[0] - ntest

X_train_depth1 = data[:testindx,np.r_[0:12,12]]
X_train_depth2 = data[:testindx,np.r_[0:12,13]]
X_train_depth3 = data[:testindx,np.r_[0:12,14]]
X_train_depth4 = data[:testindx,np.r_[0:12,15]]
X_train_depth5 = data[:testindx,np.r_[0:12,16]]
X_train_depth6 = data[:testindx,np.r_[0:12,17]]
X_train_depth7 = data[:testindx,np.r_[0:12,18]]


Y_train = data[:testindx,20]               
X_test = data[testindx:,:]
print (X_test.shape) 
#print ("shape of X_train:",X_train.shape)
print ("shape of Y_train:",Y_train.shape)

In [ ]:
# sample weights
histoAR = Y_train.copy()
nbins = 4 ### works best[4 with no sqrt / 500 with sqrt normed] / 1000 with norm*100 / 4000 ok no sqrt
true_hist = np.histogram(histoAR,bins = nbins,range=(np.min(Y_train)-0.001,np.max(Y_train)+0.001))
binweight = true_hist[0][np.digitize(histoAR,true_hist[1]) - 1]
print ("true_hist[0] = ", true_hist[0])
binweight = 1/binweight
binweight = 100*binweight/np.sum(np.unique(binweight)) ## 5000/5 ok
print("sum weight:",np.sum(np.unique(binweight)))
print("true_hist",true_hist[0])
print("binweight",binweight)
print("weight_vals:",np.unique(binweight))
plt.hist(histoAR,bins=nbins)
for i in true_hist[1]:
    plt.axvline(i,color='r')
plt.yscale("log")

In [ ]:
import keras.backend as kb
def custom_loss(preds, targets):
    return (((preds-targets)**2)/targets)

In [ ]:
PYTHONHASHSEED=0
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)

from keras.layers import LeakyReLU
from keras import optimizers
print ("creating model=========>")
model = Sequential()

model.add(Dense(40, input_shape=(X_train.shape[1],)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.1))

model.add(Dense(500))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.1))

model.add(Dense(800))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(700))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(20))
model.add(LeakyReLU(alpha=0.2))

model.add(Dense(1))

model.compile(loss=custom_loss,optimizer='adam')
model.summary()
print ("fitting now=========>")
history = model.fit(X_train,Y_train , batch_size=5000, epochs=500, validation_split=0.2,
                    verbose=1,sample_weight=binweight)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# evaluate model
preds = model.predict(X_test[:,np.r_[0:12,13]])
targets = X_test[:,20]
uncorrected = X_test[:,27]

targets = targets.reshape(targets.shape[0],1)
uncorrected = uncorrected.reshape(uncorrected.shape[0],1)

In [ ]:
from matplotlib import gridspec
%matplotlib inline
i=0

n_bins = 100

fig = plt.figure(figsize=(7,7))
gs = gridspec.GridSpec(2,1, height_ratios=[2, 1]) 
#ax = fig.add_subplot(2,1,1)
ax = plt.subplot(gs[0])
a=round(np.mean(targets[:,i]),3)
ax.hist(targets[:,i], range=(0,20), bins=100, color = 'red', label='truth no PU, Mean='+str(a),histtype='step',linewidth=1.5)

b=round(np.mean(preds[:,i]),3)
ax.hist(preds[:,i], range=(0,20), bins=100, color = 'green', label='predicted, Mean='+str(b),histtype='step',linewidth=1.5)

plt.hist(df['hcal_edepth2_un'],   range=(0,20), bins=100, color = 'yellow', label='truth PU',histtype='step',linewidth=1.5)

ax.legend(loc='upper right')
ax.set_yscale('log')
#ax.set_xscale('log')
plt.title('ieta=24', loc='left')
plt.title('Depth: 2', loc='right')
ax1 = plt.subplot(gs[1])
#ax1 = fig.add_subplot(2,1,2)
rat = (preds[:,i]/targets[:,i])

ax1.plot(rat, 'o')
ax1.axis([0, 20, -1, 2])
plt.ylabel('predicted / truth no PU')

ax1.plot([0, 20], [1, 1])

plt.savefig('Plots/Final/ieta24_Depth2_logScale.png')
plt.savefig('Plots/Final/ieta24_Depth2_logScale.pdf')
plt.show()


In [ ]:
%matplotlib inline
i=0
plt.hist(targets[:,i], bins =100, range=(0,20),color = 'red', label='truth no PU',histtype='step',linewidth=1.5)
a=round(np.mean(targets[:,i]),3)
plt.axvline(np.mean(targets[:,i]), color = 'red', linewidth=0.5, label='Mean='+str(a))

plt.hist(preds[:,i], bins =100, range=(0,20),color = 'green', label='predicted',histtype='step',linewidth=1.5)
b=round(np.mean(preds[:,i]),3)
plt.axvline(np.mean(preds[:,i]), color = 'green', linewidth=0.5, label='Mean='+str(b))

plt.hist(df['hcal_edepth2_un'], bins =100, range=(0,20),color = 'yellow', label='truth PU',histtype='step',linewidth=1.5)
c=round(np.mean(df['hcal_edepth2_un']),3)
#plt.axvline(np.mean(df['hcal_edepth2_un']), color = 'yellow', linewidth=0.5, label='Mean='+str(c))


plt.legend(loc='upper right')
plt.show()

In [ ]:
#save model
import os
if not os.path.exists('models'):
    os.makedirs('models')
model.save('models/model_ieta24_depth2.h5')